# Simultaneously Visualize Flom Decompositions in Different Directions

## Unfold a Polygon to create a Translation Surface

In [ ]:
from ipyvue_flatsurf import FlatSurface
from flatsurf import translation_surfaces, polygons, similarity_surfaces

In [ ]:
t = polygons.triangle(2, 3, 10)
B = similarity_surfaces.billiard(t)
S = B.minimal_cover('translation')
S = S.erase_marked_points()

## A Visulation for a Flow Decomposition

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(S)
D = O.decomposition((1, 0))

In [ ]:
V = FlatSurface(D)
print(D)
V

## Use the Same Glueings for Another Flow Decomposition
Clicking on half edges in the above figure forces edges to be glued. The same gluings will then be replayed in the figure below.

In [ ]:
import ipywidgets

DD = O.decomposition((0, 1))
print(DD)
W = FlatSurface(DD)
ipywidgets.link((V,'inner'), (W, 'forced'))
W

# Passing to a Deformed Surface

We deform the surface with a vector from the tangent space so this is not a billiard anymore.

In [ ]:
for decomposition in O.decompositions(64):
    O.update_tangent_space_from_flow_decomposition(decomposition)
    if O.dimension() > 2: break

tangent = O.lift(O.tangent_space_basis()[2])
length = sum(abs(x.parent().number_field(x)) for x in tangent) / len(tangent)

upper_bound = 1
while upper_bound < length:
    upper_bound *= 2
    
# Shrink the deformation somewhat more so we do not need too many flips.
upper_bound *= 64

In [ ]:
from flatsurf.geometry.pyflatsurf_conversion import from_pyflatsurf

try:
    deformation = [O.V2(x / upper_bound, x / (2*upper_bound)).vector for x in tangent]
    deformed = from_pyflatsurf((O._surface + deformation).codomain()).delaunay_triangulation()
except Exception:
    print("The deformation crosses over a vertex. Trying with a smaller deformation.")
    upper_bound *= 2
    
deformed

## Search for Interesting Directions to Visualize

In [ ]:
O = GL2ROrbitClosure(deformed)

decompositions = {}

for decomposition in O.decompositions(64):
    if len(decomposition.cylinders()) == 0:
        continue
    key = (len(decomposition.cylinders()), len(decomposition.minimal_components()))
    if key not in decompositions:
        decompositions[key] = decomposition
    if len(decompositions) >= 2:
        break

keys = list(decompositions.keys())
v0 = decompositions[keys[0]].u
v1 = decompositions[keys[1]].u

In [ ]:
D = O.decomposition(v0)
print(D)
FlatSurface(D)

In [ ]:
DD = O.decomposition(v1)
print(DD)
FlatSurface(DD)

## Making the Search Directions Orthogonal

In [ ]:
A = matrix([v0, v1])
E = matrix([[1, 0], [0, 1]])
N = E * ~A.transpose()

In [ ]:
T = deformed.apply_matrix(N, in_place=False)
T = T.delaunay_triangulation()

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(T)
D = O.decomposition((E[0]))
print(D)
V = FlatSurface(D)
V

In [ ]:
O.decomposition((E[1]))

Pass to a triangulation that is aligned with the cylinders.

In [ ]:
T = from_pyflatsurf(D.decomposition.triangulation())
T

Currently, the minimal components are triangulated in a way that respects their internal structure, see https://github.com/flatsurf/flatsurf/issues/272. This leads to very stretched out pictures.
We Delaunay triangulate the interior of such components to make these components shorter.

In [ ]:
O = GL2ROrbitClosure(T)
D = O.decomposition(E[0])
S = D.decomposition.surface()
noncylinders = D.minimal_components() + D.undetermined_components()

inners = []

for component in noncylinders:
    perimeter = [connection.saddleConnection().source() for connection in component.perimeter()]
    
    for p in component.perimeter():
        connection = p.saddleConnection()
        assert connection == type(connection)(S, connection.source())
        inner = connection.source()
        if not p.boundary():
            inners.append(inner)
            inners.append(-inner)
        while True:
            inner = S.nextAtVertex(inner)
            if inner in perimeter or -inner in perimeter:
                break
            inners.append(inner)
            inners.append(-inner)
    
while True:
    for inner in inners:
        import pyflatsurf
        if S.delaunay(inner) == 0:
            S.flip(inner)
            break
    else:
        break
        
T = from_pyflatsurf(S)
T

In [ ]:
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(T)
D = O.decomposition(E[0])
print(D)
D

In [ ]:
DD = O.decomposition(E[1])
print(DD)
DD

Now the pictures might be very squeezed, so try to rescale to make things reasonable again.

In [ ]:
ratios = [float(c.width() / c.height()) for c in D.cylinders()] + [float(c.height() / c.width()) for c in DD.cylinders()]
print(f"Old aspect ratio of cylinders: {ratios}")

# We try to move everything simultaneously close to a 4:3 aspect ratio.
target = float(4 / 3)

# The factor we would have to multiply each cylinder with to make it 4:3.
distance = [target / ratio for ratio in ratios]

# We average over these factors.
scale = exp(sum([log(d) for d in distance]) / len(distance))

# Turn the above factor into something that lives in the number field.
scale = ZZ(ceil(scale * 1000)) / 1000
ratios = [scale * ratio for ratio in ratios]

print(f"New aspect ratio of cylinders: {ratios}")
A = matrix([[scale, 0], [0, 1]])
U = T.apply_matrix(A, in_place=False)

Glue cylinders in horizontal direction.

In [ ]:
O = GL2ROrbitClosure(U)
D = O.decomposition((E[0]))
print(D)
V = FlatSurface(D)

forced = []

surface = D.decomposition.surface()
for cylinder in D.cylinders():
    for connection in cylinder.perimeter():
        connection = connection.saddleConnection()
        assert connection == type(connection)(surface, connection.source())
        
    perimeter = [connection.saddleConnection().source() for connection in cylinder.perimeter()]
    inners = []
    
    for connection in cylinder.perimeter():
        inner = connection.saddleConnection().source()
        if not connection.vertical():
            inners.append(inner)
        while True:
            inner = surface.nextAtVertex(inner)
            if inner in perimeter or -inner in perimeter:
                break
            inners.append(inner)

    # Force all inner edges except for the one least aligned to the flow direction.
    def dot(he):
        v = surface.fromHalfEdge(he)
        return abs(float(v.x()) * float(E[0][0]) + float(v.y()) * float(E[0][1]))
        
    smallest = inners[0]
    
    for inner in inners:
        if dot(inner) < dot(smallest):
            smallest = inner

    forced.extend(list(set([he.id() for he in inners if he != smallest and he != -smallest])))
        
V.forced = forced
V

In [ ]:
D = O.decomposition((E[1]))
W = FlatSurface(D)
ipywidgets.link((V,'inner'), (W, 'forced'))
print(D)
W